In [5]:
import math
import os
import numpy as np
from sklearn.metrics import f1_score
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, f1_score, confusion_matrix


In [6]:
# Loading the custom fruit freshness model
model = load_model('fruit_freshness_model.h5')

# Display the model's architecture
model.summary()


Model: "model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_1 (InputLayer)                 │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 15,467,360 (59.00 MB)

 Trainable params: 752,670 (2.87 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

 Optimizer params: 2 (12.00 B)

In [7]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import f1_score

# Directory for test data
test_dir = r'Data/test'  

# Image Data Generator for test data (rescale)
test_datagen = ImageDataGenerator(rescale=1./255)  # Normalize pixel values

# Load the test images with categorical class mode
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),  
    batch_size=32,           
    class_mode='categorical',  
    shuffle=False            
)  

# Predict on the test set
y_pred_prob = model.predict(test_generator)
y_pred = np.argmax(y_pred_prob, axis=1)  # Convert predictions to class indices

# Get the true labels
y_true = test_generator.classes

# Calculate F1 Score
f1 = f1_score(y_true, y_pred, average='weighted')
print(f'Weighted F1 Score: {f1:.4f}')


Found 3846 images belonging to 30 classes.


C:\Users\Prinshu Kumar\anaconda3\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 79/121 ━━━━━━━━━━━━━━━━━━━━ 3:40 5s/step

C:\Users\Prinshu Kumar\anaconda3\lib\site-packages\PIL\Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


121/121 ━━━━━━━━━━━━━━━━━━━━ 649s 5s/step
Weighted F1 Score: 0.6709


# Displaying classification report (precision, recall, F1-score for each class)


In [8]:
# Displaying classification report (precision, recall, F1-score for each class)
class_labels = list(test_generator.class_indices.keys())
print(classification_report(y_true, y_pred, target_names=class_labels))

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print('Confusion Matrix:')
print(conf_matrix)

                             precision    recall  f1-score   support

                  Bad Apple       0.80      0.33      0.46       126
                 Bad Banana       0.72      0.98      0.83       127
             Bad Bellpepper       0.92      0.31      0.46       155
               Bad Cucumber       0.91      0.88      0.89        81
                 Bad Grapes       0.62      0.86      0.72       126
     Bad Indian Green Chile       0.88      0.92      0.90       116
                  Bad Mango       0.92      0.85      0.88       126
                 Bad Orange       0.66      0.95      0.78       132
                 Bad Potato       0.94      0.75      0.83       116
                 Bad Tomato       0.70      0.80      0.75       148
                Fresh Apple       0.86      0.13      0.22       292
               Fresh Banana       0.77      0.87      0.82       150
           Fresh Bellpepper       0.34      0.87      0.49        86
             Fresh Cucumber      

In [9]:
import numpy as np
import tensorflow as tf
from PIL import Image
import os


model_path = 'fruit_freshness_model.h5'  
model = load_model(model_path)

# Define the class labels (update these as per your model)
class_labels = ['Bad Apple', 'Bad Banana', 'Bad Bellpepper', 'Bad Cucumber', 'Bad Grapes', 
                'Bad Indian Green Chile', 'Bad Mango', 'Bad Orange', 'Bad Potato', 'Bad Tomato', 
                'Fresh Apple', 'Fresh Banana', 'Fresh Bellpepper', 'Fresh Cucumber', 'Fresh Grapes', 
                'Fresh Indian Green Chile', 'Fresh Mango', 'Fresh Orange', 'Fresh Potato', 'Fresh Tomato', 
                'Moderate Apple', 'Moderate Banana', 'Moderate Bellpepper', 'Moderate Cucumber', 
                'Moderate Grapes', 'Moderate Indian Green Chile', 'Moderate Mango', 'Moderate Orange', 
                'Moderate Potato', 'Moderate Tomato']

# Function to load an image and make predictions
def predict_image(image_path):
    if not os.path.exists(image_path):
        print(f"Error: File not found at {image_path}")
        return None
    
    try:
        # Load and preprocess image
        img = Image.open(image_path)
        img_resized = img.resize((224, 224))  # Resize to match VGG16's input shape
        img_array = np.array(img_resized) / 255.0 
        img_array = np.expand_dims(img_array, axis=0)  

       
        predictions = model.predict(img_array)
        predicted_class = class_labels[np.argmax(predictions)]

        return predicted_class
    except Exception as e:
        print(f"Error in predicting image: {e}")
        return None


image_path = 'data/test/Bad Apple/6100533069082640041.jpg' 
predicted_class = predict_image(image_path)
if predicted_class:
    print(f'Predicted Class: {predicted_class}')
else:
    print("Prediction failed.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 710ms/step
Predicted Class: Bad Apple


In [3]:
def calculate_rate_constant(Ea,A, T):
    R = 8.314  # Universal gas constant in J/(mol*K)
   
    k = A * math.exp(-Ea / (R * T))
    return k

def predict_shelf_life(Ea,A,T):
    
    k = calculate_rate_constant(Ea,A, T)
    shelf_life = 1 / k
    return shelf_life
PreExpontialFactor ={'Fresh Apple': 645092.1348576404, 'Fresh Banana': 15370.169119416576, 'Fresh Bell Pepper': 23330.754782432436, 'Fresh Cucumber': 277.94296916110966, 'Fresh Grapes': 23330.754782432436, 'Fresh Indian Green Chile': 23330.754782432436, 'Fresh Mango': 277.94296916110966, 'Fresh Orange': 5.995771324920545, 'Fresh Potato': 295.3274908449173, 'Fresh Tomato': 15370.169119416576, 'Moderate Apple': 15370.169119416576, 'Moderate Banana': 11139.41897966447, 'Moderate Bell Pepper': 113.07685698928096, 'Moderate Cucumber': 3494.564827089312, 'Moderate Grapes': 440004.8432500969, 'Moderate Indian Green Chile': 113.07685698928096, 'Moderate Mango': 208.10538328031532, 'Moderate Orange': 277.94296916110966, 'Moderate Potato': 1747.282413544656, 'Moderate Tomato': 440004.8432500969}
AcivationEnergy= {'Fresh Apple': 38867.22294049396, 'Fresh Banana': 27885.777266933837, 'Fresh Bell Pepper': 29754.351047873723, 'Fresh Cucumber': 18772.9053743136, 'Fresh Grapes': 29754.351047873723, 'Fresh Indian Green Chile': 29754.351047873723, 'Fresh Mango': 18772.9053743136, 'Fresh Orange': 10981.445673560118, 'Fresh Potato': 20641.479155253488, 'Fresh Tomato': 27885.777266933837, 'Moderate Apple': 27885.777266933837, 'Moderate Banana': 24816.431036967108, 'Moderate Bell Pepper': 15156.397555273734, 'Moderate Cucumber': 22947.857256027222, 'Moderate Grapes': 33929.302929587335, 'Moderate Indian Green Chile': 15156.397555273734, 'Moderate Mango': 16191.564734321395, 'Moderate Orange': 18772.9053743136, 'Moderate Potato': 22947.857256027222, 'Moderate Tomato': 33929.302929587335}

PreExpontialFactor

{'Fresh Apple': 645092.1348576404,
 'Fresh Banana': 15370.169119416576,
 'Fresh Bell Pepper': 23330.754782432436,
 'Fresh Cucumber': 277.94296916110966,
 'Fresh Grapes': 23330.754782432436,
 'Fresh Indian Green Chile': 23330.754782432436,
 'Fresh Mango': 277.94296916110966,
 'Fresh Orange': 5.995771324920545,
 'Fresh Potato': 295.3274908449173,
 'Fresh Tomato': 15370.169119416576,
 'Moderate Apple': 15370.169119416576,
 'Moderate Banana': 11139.41897966447,
 'Moderate Bell Pepper': 113.07685698928096,
 'Moderate Cucumber': 3494.564827089312,
 'Moderate Grapes': 440004.8432500969,
 'Moderate Indian Green Chile': 113.07685698928096,
 'Moderate Mango': 208.10538328031532,
 'Moderate Orange': 277.94296916110966,
 'Moderate Potato': 1747.282413544656,
 'Moderate Tomato': 440004.8432500969}

In [4]:
AcivationEnergy

{'Fresh Apple': 38867.22294049396,
 'Fresh Banana': 27885.777266933837,
 'Fresh Bell Pepper': 29754.351047873723,
 'Fresh Cucumber': 18772.9053743136,
 'Fresh Grapes': 29754.351047873723,
 'Fresh Indian Green Chile': 29754.351047873723,
 'Fresh Mango': 18772.9053743136,
 'Fresh Orange': 10981.445673560118,
 'Fresh Potato': 20641.479155253488,
 'Fresh Tomato': 27885.777266933837,
 'Moderate Apple': 27885.777266933837,
 'Moderate Banana': 24816.431036967108,
 'Moderate Bell Pepper': 15156.397555273734,
 'Moderate Cucumber': 22947.857256027222,
 'Moderate Grapes': 33929.302929587335,
 'Moderate Indian Green Chile': 15156.397555273734,
 'Moderate Mango': 16191.564734321395,
 'Moderate Orange': 18772.9053743136,
 'Moderate Potato': 22947.857256027222,
 'Moderate Tomato': 33929.302929587335}

Enter The Temp in degree celcius7
Fruit: Fresh Apple
Shelf Life: 27.385899404362036


In [3]:
import numpy as np
import tensorflow as tf
from PIL import Image
import os
import math

# Load your trained model (make sure the path is correct)
model_path = 'fruit_freshness_resnet_model.h5'  # Ensure the model file has the correct extension
model = load_model(model_path)

# Define the class labels (update these as per your model)
class_labels = ['Bad Apple', 'Bad Banana', 'Bad Bellpepper', 'Bad Cucumber', 'Bad Grapes', 
                'Bad Indian Green Chile', 'Bad Mango', 'Bad Orange', 'Bad Potato', 'Bad Tomato', 
                'Fresh Apple', 'Fresh Banana', 'Fresh Bellpepper', 'Fresh Cucumber', 'Fresh Grapes', 
                'Fresh Indian Green Chile', 'Fresh Mango', 'Fresh Orange', 'Fresh Potato', 'Fresh Tomato', 
                'Moderate Apple', 'Moderate Banana', 'Moderate Bellpepper', 'Moderate Cucumber', 
                'Moderate Grapes', 'Moderate Indian Green Chile', 'Moderate Mango', 'Moderate Orange', 
                'Moderate Potato', 'Moderate Tomato']

# Arrhenius equation-related factors
PreExpontialFactor = {
    'Fresh Apple': 645092.1348576404, 'Fresh Banana': 15370.169119416576, 'Fresh Bellpepper': 23330.754782432436, 
    'Fresh Cucumber': 277.94296916110966, 'Fresh Grapes': 23330.754782432436, 'Fresh Indian Green Chile': 23330.754782432436, 
    'Fresh Mango': 277.94296916110966, 'Fresh Orange': 5.995771324920545, 'Fresh Potato': 295.3274908449173, 
    'Fresh Tomato': 15370.169119416576, 'Moderate Apple': 15370.169119416576, 'Moderate Banana': 11139.41897966447, 
    'Moderate Bell Pepper': 113.07685698928096, 'Moderate Cucumber': 3494.564827089312, 'Moderate Grapes': 440004.8432500969, 
    'Moderate Indian Green Chile': 113.07685698928096, 'Moderate Mango': 208.10538328031532, 'Moderate Orange': 277.94296916110966, 
    'Moderate Potato': 1747.282413544656, 'Moderate Tomato': 440004.8432500969
}

AcivationEnergy = {
    'Fresh Apple': 38867.22294049396, 'Fresh Banana': 27885.777266933837, 'Fresh Bellpepper': 29754.351047873723, 
    'Fresh Cucumber': 18772.9053743136, 'Fresh Grapes': 29754.351047873723, 'Fresh Indian Green Chile': 29754.351047873723, 
    'Fresh Mango': 18772.9053743136, 'Fresh Orange': 10981.445673560118, 'Fresh Potato': 20641.479155253488, 
    'Fresh Tomato': 27885.777266933837, 'Moderate Apple': 27885.777266933837, 'Moderate Banana': 24816.431036967108, 
    'Moderate Bell Pepper': 15156.397555273734, 'Moderate Cucumber': 22947.857256027222, 'Moderate Grapes': 33929.302929587335, 
    'Moderate Indian Green Chile': 15156.397555273734, 'Moderate Mango': 16191.564734321395, 'Moderate Orange': 18772.9053743136, 
    'Moderate Potato': 22947.857256027222, 'Moderate Tomato': 33929.302929587335
}

# Arrhenius equation: calculates the rate constant 'k' and shelf life
def calculate_rate_constant(Ea, A, T):
    R = 8.314  # Universal gas constant in J/(mol*K)
    k = A * math.exp(-Ea / (R * T))
    return k

def predict_shelf_life(Ea, A, T):
    k = calculate_rate_constant(Ea, A, T)
    shelf_life = 1 / k
    return shelf_life

# Function to load an image and make predictions
def predict_image(image_path):
    if not os.path.exists(image_path):
        print(f"Error: File not found at {image_path}")
        return None
    
    try:
        # Load and preprocess image
        img = Image.open(image_path)
        img_resized = img.resize((224, 224))  # Resize to match VGG16's input shape
        img_array = np.array(img_resized) / 255.0  # Normalize pixel values
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

        # Make prediction
        predictions = model.predict(img_array)
        predicted_class = class_labels[np.argmax(predictions)]

        return predicted_class
    except Exception as e:
        print(f"Error in predicting image: {e}")
        return None

# Main flow
image_path = 'apple.jpg'  # Update this path to your image
predicted_class = predict_image(image_path)

if predicted_class:
    print(f'Predicted Class: {predicted_class}')
    
    if predicted_class in ['Bad Apple', 'Bad Banana', 'Bad Bellpepper', 'Bad Cucumber', 'Bad Grapes', 
                           'Bad Indian Green Chile', 'Bad Mango', 'Bad Orange', 'Bad Potato', 'Bad Tomato']:
        print("Shelf Life: 0 ")
    else:
        # Get temperature input
        T = float(input("Enter the temperature in degrees Celsius: ")) + 273.15  # Convert Celsius to Kelvin

        # Calculate shelf life for fresh or moderate fruits
        Ea = AcivationEnergy[predicted_class]
        A = PreExpontialFactor[predicted_class]
        shelf_life = predict_shelf_life(Ea, A, T)

        print(f"Shelf Life for {predicted_class}: {shelf_life:.2f} days")
else:
    print("Prediction failed.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted Class: Bad Indian Green Chile
Shelf Life: 0 
